# First model for warehouse picking optimisation

This notebook contains code to optimise the following problem.

Suppose we have a warehouse with $n$ unique products, and $n$ pick faces ($n$ must be even). The pick faces are laid out as below, in a single aisle on shelves with no shelves on top of each other, with odd number increasing on the left, and even numbers increasing from bottom up on the right. The diagram is viewed from the air, looking down, and let's say up is "north".

```
+---+    >.....v    +---+
|n-1|    :     :    | n |
+---+    ^.>...v    +---+
|   |    :     :    |   |

           ...

|   |    :     :    |   |
+---+    ^.>...v    +---+
| 7 |    :     :    | 8 |
+---+    ^.>...v    +---+
| 5 |    :     :    | 6 |
+---+    ^.>...v    +---+
| 3 |    :     :    | 4 |
+---+    ^.>...v    +---+
| 1 |    :     :    | 2 |
+---+    .     v    +---+
     @...^     :
 PC  #.<       >..Sorting
       :         v
       ^.........<
```

The picking procedure (a single "pick wave") works as follows:

1. The picker starts at the PC, marked with an at sign (`@`) on the map
2. The picker prints out a flow path from the WMS+optimiser at the computer (`PC`)
3. The picker then starts walking up on the left hand side of the aisle, next to the odd pick faces.
4. If the picker has nothing to pick north of the current location, they can move to the right and start picking products in even pick faces, and walk south along the right side of the aisle.
5. Once the picker has picked all items from their shelves, they move down to the sorting facility (`Sorting` on the diagram)
6. The picker sorts and packs all orders into their correct shipping packages, and drops them off at the same location
7. The picker moves back to the PC and finishes at the hash sign (`#`) on the map

## The cost structure

We model the time taken to pick an order as the cost for this problem.

It's relatively difficult to estimate the actual time taken to complete each step of a pick wave, so we leave this in a highly modular and customisable system.

We basically assume the cost of a pick wave is a linear function of the following steps which may have more difficult, non-linear cost functions:

* $c_{\text{fixed}}$: A fixed cost of one pick wave (e.g. printing, moving from computer to first shelf, last shelf to sorting, sorting to computer, rest)
* $c_{\text{move}}$: Moving one shelf up or down the aisle (note that there are always an even number of these steps)
* $c_{\text{across}}$: Moving across the aisle from left to right
* $c_{\text{picking}}$: Picking objects from a shelf into containers, this is encoded as $p_1$, $p_2$, $\dots$, $p_q$ where $q$ is the number of containers, and $p_i$ is the number of identical objects picked form a shelf into container $i$. The cost for this tuple is a function $c_{\text{picking}}(q, (p_1, p_2, \dots, p_q))$.
* $c_{\text{sorting}}$: Sorting objects in $q$ containers to $r$ shipping "packages" (note the terminology: containers are used for picking from shelves, packages for final postable boxes), this is encoded as $p_1$, $p_2$, $\dots$, $p_q$ for the number of items in teh containers and $g_1$, $g_2$, $\dots$, $g_r$ for the number of items in the final packages, so the cost function is $c_{\text{sorting}}(q, r, (p_1, p_2, \dots, p_q), (g_1, g_2, \dots, g_r))$

The total cost, $C:=c_{\text{total}}$, is then given by the sum of these costs:

$$
C=n_{\text{fixed}}\times c_{\text{fixed}}+n_{\text{move}}\times c_{\text{move}}+n_{\text{across}}\times c_{\text{across}}+\sum_{q,p_1,p_2,\dots,p_q}n_{\text{picking}}(q, (p_1, p_2, \dots, p_q))\times c_{\text{picking}}(q, (p_1, p_2, \dots, p_q))+\sum_{q,r,p_1,p_2,\dots,p_q,g_1,g_2,\dots,g_r}n_{\text{sorting}}(q, r, (p_1, p_2, \dots, p_q), (g_1, g_2, \dots, g_r))\times c_{\text{sorting}}(q, r, (p_1, p_2, \dots, p_q), (g_1, g_2, \dots, g_r))
$$

Where the $n_{\text{fixed}}$, $n_{\text{move}}$, and $n_{\text{across}}$ are the number of pick waves, the number of times a picker moved up or down, and the number of times a picker moved across the aisle, respectively.

$n_{\text{picking}}(q, (p_1, p_2, \dots, p_q))$ is the number of times the same type of object was picked from a shelf into $q$ containers with the $q$ containers getting $p_1$, $p_2$, \dots$, $p_q$ items per container.

$n_{\text{sorting}}(q, r, (p_1, p_2, \dots, p_q), (g_1, g_2, \dots, g_r))$ is the number of times that objects in $q$ containers with $p_i$ objects in container $i$ were sorted into $r$ packages with $g_i$ objects in package $i$.

Note a possible confusion of terminology: even though the notation $p_1$, $p_2$, $\dots$, $p_q$ is used for both picking into $q$ containers as well as sorting from these containers, these numbers are not the same. The latter is the sum of all objects picked into the container from all shelves, whereas the former is picking one of the same kind of object.

In [ ]:
from random import shuffle
from scipy.stats import zipf
from numpy.random import choice

In [ ]:
# number of pick faces
no_pick_faces = 10

# zipf's law shape parameter
zipf_shape = 1.1

In [ ]:
def generate_product_probabilities_pareto(zipf_shape, products):
    # to get wikipedia zipf from scipy zipf, just truncate
    distribution = zipf(zipf_shape)
    freqs = [distribution.pmf(n+1) for n in range(len(products))]
    freqs = freqs / sum(freqs)
    shuffle(freqs)

    return dict(zip(products, freqs))

In [ ]:
# assignment of products to pick faces
def assign_products_to_pick_faces_random(products, pick_faces):
    # random
    pick_face_assignments = pick_faces.copy()
    shuffle(pick_face_assignments)

    product_to_pick_face = dict(zip(products, pick_face_assignments))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

def assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities):
    # according to freq (akin to Andrew's pareto method)
    product_probs_for_assignment = list(product_probabilities.items())
    product_probs_for_assignment.sort(key=lambda x: -x[1])
    product_to_pick_face = dict(zip(map(lambda x: x[0], product_probs_for_assignment), pick_faces))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

In [ ]:
def generate_new_order(number_of_products, product_probabilities):
    labels, weights = zip(*product_probabilities.items())
    return list(choice(labels, number_of_products, p=weights))

In [ ]:
orders = [generate_new_order(5, product_probabilities) for n in range(50)]

In [ ]:
# the pick face labels
pick_faces = ["face {}".format(n+1) for n in range(no_pick_faces)]

# there's the same number of products as pick faces
products = ["product {}".format(n+1) for n in range(no_pick_faces)]

product_probabilities = generate_product_probabilities_pareto(zipf_shape, products)
product_to_pick_face, pick_face_to_product = assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities)

pick_faces, products, product_probabilities, product_to_pick_face, pick_face_to_product